In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import pywt
import itertools
from scipy import stats
import pickle

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


In [132]:
scoreType = 'LIX'
#scoreType = 'SumOfScores'
#scoreType = 'Difficulty'
#scoreType = 'Complexity'

subj_session_name = 'sa\\Test_woChinRest'

subj_session_name_fig = subj_session_name
#subj_session_name_fig = subj_session_name + '_p1'
#subj_session_name_fig = subj_session_name + '_p2'


In [109]:
# open the interactive figures 


figx = pickle.load(open(r'C:\DTU\Results\201812_PilotExptForMovement\pupil Size\Complete pupil size' + '\\' + subj_session_name_fig, 'rb'))
figx2 = pickle.load(open(r'C:\DTU\Results\201812_PilotExptForMovement\pupil Size\Trial by trial' + '\\' + subj_session_name_fig + '_relativeComplete_allTrials_wAvg' + '_' + scoreType, 'rb'))

figx.show() # Show the figure, edit it, etc.!
figx2.show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\DTU\\Results\\201812_PilotExptForMovement\\pupil Size\\Trial by trial\\rh\\Test_woChinRest_relativeComplete_allTrials_wAvg_LIX'

In [79]:
relative_difference = False # True means computing relative; False means in %
absoluteSize = False # True means absolute, False means relative

In [80]:
TimeDwellOrig = 800
TimeFixation = 300

In [81]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New

In [82]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [83]:
def DwellTimeForBaseline(UserKeys_wDwellTime):
    
    DwellTime = list()
    
    for key in UserKeys_wDwellTime:
        if key[1] == 'NextPhrase':
            #print('NextPhrase found at ', key[2])
            if key[2] == 1:
                DwellTime.append(key[3])
                
    return DwellTime

In [84]:
def OptiKeyTypingTime(userKeys):
    
    timeTyping = dict()
    
    time1, t1, t2 = userKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = userKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    timeTyping['startTime'] = startTime
    timeTyping['endTime'] = endTime
    
    return timeTyping

In [85]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [86]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [87]:
def Combine2ColumnsTo1GazeLog(GazeLog, Column_1, Column_2):
    
    JoinedList = list()
    
    Column_beforeDecimal = [item4[Column_1] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    Column_afterDecimal = [item4[Column_2] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    
    
    for i in range(0, len(Column_beforeDecimal)):
        if 'Valid' not in Column_beforeDecimal[i] and 'Valid' not in Column_afterDecimal[i]:
            if 'nan' not in Column_beforeDecimal[i] and 'nan' not in Column_afterDecimal[i]:
                JoinedList.append(float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]))
            else:
                JoinedList.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            JoinedList.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
            
    return JoinedList

In [88]:
def Convert2ColumnsToFormPupilSizes(GazeLog):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    PupilLogL = Combine2ColumnsTo1GazeLog(GazeLog, -5, -4)
    PupilLogR = Combine2ColumnsTo1GazeLog(GazeLog, -2, -1)
            
    # if one of the pupils are nan, the other one is converted too
    nPupil = -1
    for pupilL in PupilLogL:
        nPupil = nPupil + 1
        if np.isnan(pupilL):
            if nPupil < len(PupilLogR):
                if not np.isnan(PupilLogR[nPupil]):
                    PupilLogR[nPupil] = np.nan
            else:
                del PupilLogL[len(PupilLogR):]
                
    nPupil = -1
    for pupilR in PupilLogR:
        nPupil = nPupil + 1
        if np.isnan(pupilR):
            if nPupil < len(PupilLogL):
                if not np.isnan(PupilLogL[nPupil]):
                    PupilLogL[nPupil] = np.nan
            else:
                del PupilLogR[len(PupilLogL):]
                
    #print(len(PupilLogL), len(PupilLogR))
    
    return PupilLogL, PupilLogR

In [89]:
def FindTrialEndTimes(KeysSelected, timeTyping):
    # function to find start and end of tasks in experiments
    timeStartEnd = list() # format of this list will be: [startTime1, endTime1/startTime2, endTime2/startTime3, ..., endTimeN]
    
    timeStartEnd.append(timeTyping['startTime'])
    
    nTrial = 1
    
    for keys in KeysSelected:
        
        if keys[1] == 'NextPhrase':
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            timeStartEnd.append(endTimeTrial)
    
    
    timeStartEnd.append(timeTyping['endTime'])
    
    
    return timeStartEnd

In [90]:
def CreateTimeEpochsOfTrials(TimeStartEndMixed, UserKeys):
    # function to use list of mixed start and end times of trials and keys looked at by user to create trial epochs
    
    TimeEpochTrial = dict()
    TimeEpochTrial['Start'] = list()
    TimeEpochTrial['End'] = list()
    
    # Create list of times in userKeys to be able to use function 'nearestTimePoint'
    UserKeysStrTimes = [item3[0] for item3 in UserKeys]
    UserKeysTimes = timeConversion(UserKeysStrTimes)
    
    Flag_FoundSleepKey = 0 # Flag to indicate finding sleep key
    
    n = -1
    for time in TimeStartEndMixed:
        n = n + 1
        Flag_FoundSleepKey = 0
        
        if n == 0: # first time is only start time for the first trial
            TimeEpochTrial['Start'].append(time)
            continue
        elif n == len(TimeStartEndMixed)-1: # last time is only the end time for last trial
            
            TimeEpochTrial['End'].append(time)
            
        else: # the middle elements need to be divided into start and end
            TimeEpochTrial['End'].append(time)
            
            timeCheck = time
            
            # find the time in userkeys. Keep going to the previous element till you reach start of selection of
            # nextPhrase key
            while Flag_FoundSleepKey < 1:
                
                nearestToTrialStartTime, nearestToTrialStartInd = nearestTimePoint(UserKeysTimes, timeCheck)
                indCheck = nearestToTrialStartInd
                
                if 'NextPhrase' not in UserKeys[indCheck][1]:
                    TimeEpochTrial['Start'].append(nearestToTrialStartTime)
                    Flag_FoundSleepKey = 1
                    break
                else:
                    indCheck = indCheck - 2 # 2 added instead of 1, to allow nearestTimePoint to find the one before this
                    timeCheck = UserKeysTimes[indCheck]
                    
                
    return TimeEpochTrial      
            

In [91]:
def DwellTimeForBaseline(UserKeys_wDwellTime):
    
    DwellTime = list()
    
    for key in UserKeys_wDwellTime:
        if key[1] == 'NextPhrase':
            #print('NextPhrase found at ', key[2])
            if key[2] == 1:
                DwellTime.append(key[3])
                
    return DwellTime

In [92]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    

    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [93]:
def PupilSizeFromTrialTimes(TimeTrial, TimeGazeLog, TimeInternalGazeLog, PupilSizeLogL, PupilSizeLogR):
    # find pupil sizes from the start and end time given
    
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(TimeGazeLog, TimeTrial[0])
    timeEnd, timeEndInd = nearestTimePoint(TimeGazeLog, TimeTrial[1])
    
    pupilSize_TrialL = PupilSizeLogL[timeStartInd: timeEndInd]
    pupilSize_TrialR = PupilSizeLogR[timeStartInd: timeEndInd]
    
    TimeInternal_Trial = TimeInternalGazeLog[timeStartInd: timeEndInd]
    
    TimeGaze_Trial = TimeGazeLog[timeStartInd: timeEndInd]
    
    return pupilSize_TrialL, pupilSize_TrialR, TimeGaze_Trial, TimeInternal_Trial

In [94]:
def filterBlinks(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (23 samples) before and after the blink will also be removed
    extraBlinkSamples = 23    
    
    # remove single missing data, that are due to hardware error
    missingVal_Single = np.argwhere(np.isnan(pupilData))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list
    missingVal_SingleDifference = [t - s for s, t in zip(missingVal_Single, missingVal_Single[1:])] # find difference 
    # between consecutive elements
    missingVal_SingleDifference.insert(0, missingVal_Single[0]) # insert the first blink index in the beginning of list
    
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index to 
    # the next nan value
    
    # first remove the single nan values, which are missing data
    eyeTracker_missingData = list() # list with index of single missing data  
    valInd = -1

    for val in missingVal_SingleDifference:
        valInd = valInd + 1
        if valInd == 0:
            continue
        if val != 1:
            if missingVal_SingleDifference[valInd-1] !=1: # if there are 2 consecutive missing values (denoted by 2 consecutive
                # non 1 numbers, they are added to the list of eyeTracker_missingData)
                eyeTracker_missingData.append(sum(missingVal_SingleDifference[:valInd]))
                
    # remove single missing values from pupil data
    pupilData_woSingleMissingData0 = [pupilData[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(pupilData))]
    pupilData_woSingleMissingData = [x for x in pupilData_woSingleMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woSingleMissingData0 = [timeListComplete[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(timeListComplete))]
    timeList_woSingleMissingData = [x for x in timeList_woSingleMissingData0 if x]
    
#     print(len(timeList_woSingleMissingData))
    
    
    
    
    # find the nan values again from pupilData_woSingleMissingData
    missingVal_Rest = np.argwhere(np.isnan(pupilData_woSingleMissingData))
    missingVal_Rest = list(itertools.chain.from_iterable(missingVal_Rest))
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    
    # compile and create list of start and end of blinks
    blink_missingData = dict()
    blink_missingData['Start'] = list()
    blink_missingData['End'] = list()
    
    valInd = -1
    for val in missingVal_RestDifference:
        valInd = valInd + 1
        if val > 1:
            #print('value', val)
            # instead of appending the actual index of blink start, since 250ms before and after the blink need to be
            # removed, it is also appended here.
            
            # just make sure that the additional samples do not make the index of blink go in negative
            if sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples > 0:
                
                blink_missingData['Start'].append(sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples)
            else:
                blink_missingData['Start'].append(0)
            
            if valInd == 0:
                continue
                
            # make sure that the additional samples do not increase the index to more than the length of the pupilData
            if sum(missingVal_RestDifference[:valInd])+extraBlinkSamples < len(pupilData_woSingleMissingData):
                blink_missingData['End'].append(sum(missingVal_RestDifference[:valInd])+extraBlinkSamples)
            else:
                blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
            #print('end', sum(missingVal_RestDifference[:valInd]))
      
    # add the last blink index
    # make sure that the additional samples do not increase the index to more than the length of the pupilData
    if sum(missingVal_RestDifference)+extraBlinkSamples < len(pupilData_woSingleMissingData):
        blink_missingData['End'].append(sum(missingVal_RestDifference)+extraBlinkSamples)
    else:
        blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
      
    
    # print start and end values
    #for ind in range(0,len(blink_missingData['Start'])):
    #    print(blink_missingData['Start'][ind]+23, blink_missingData['End'][ind]-23)
        
        
        
    # need to create a list containing indexes that are to be removed
    blinkIndexList = list()
    
    #print(len(blink_missingData['Start']), len(blink_missingData['End']))
    
    
    # remove blinks and additional data from pupil data to get filtered data
    for indInd in range(0, len(blink_missingData['Start'])):
        blinkIndexList.append(range(blink_missingData['Start'][indInd], blink_missingData['End'][indInd]+1))
    # flatten the list
    blinkIndexList = list(itertools.chain.from_iterable(blinkIndexList))
    
    
    ##print(len(pupilData_woSingleMissingData))
    
    pupilData_woRestMissingData0 = [pupilData_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(pupilData_woSingleMissingData))]
    #for i in enumerate(pupilData_woRestMissingData0):
    #    print(i)
    pupilData_filter = [x for x in pupilData_woRestMissingData0 if x]
    
    #for i in enumerate(pupilData_filter):
    #    print(i)
        
    # remove the times for single missing values in pupil data
    timeList_woRestMissingData0 = [timeList_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(timeList_woSingleMissingData))]
    time_filter = [x for x in timeList_woRestMissingData0 if x]
    
    #print(len(pupilData_filter))
        
    if np.nan in pupilData_filter:
        print('nan values still present in pupil data')
        #for i in enumerate(pupilData_woSingleMissingData):
        #    print(i)
        
    
    return pupilData_filter, time_filter, blink_missingData

In [129]:
def plotRelativePupilSize_allTrials_wAvg(pupilDataL_List, pupilDataR_List, pupilAvgL_List, pupilAvgR_List, TrialSamples, BaselineSamples, scoreDifficultyList, subjName):
    
    #dataLenEqualizer = min(min(len(pupilData['Left']), len(pupilData['Right'])), len(timeData))
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    time = np.arange(0, len(pupilDataL_List)/90, (1/90))
    ax.plot(time, pupilDataL_List)
        #ax.plot(pupilDataL_List[i], 'b', label = str(scoreDifficultyList[i]))
    
    ax.plot(time, pupilAvgL_List)
    
    n_trial = -1
    for baselineSample in BaselineSamples:
        n_trial = n_trial + 1
        ax.plot(baselineSample/90, pupilAvgL_List[baselineSample], 'o', label = scoreDifficultyList[n_trial])
    
    plt.legend()
    
    if relative_difference:
        ax.set_ylabel('Relative pupil size [in mm]')
    else:
        ax.set_ylabel('Relative pupil size [%]')
    
    ax.set_xlabel('Time [in s]')
    #ax.set_ylim([0.8, 1.2])
    ax.set_title(subjName)
    pickle.dump(fig, open(r'C:\DTU\Results\201812_PilotExptForMovement\pupil Size\Trial by trial' + '\\' + subj_session_name_fig + '_relativeComplete_allTrials_wAvg' + '_' + scoreType, 'wb'))



In [130]:
def pupilSize_CompleteSignal_wRelativeAvgOfTrial(GazeLog, TimeEpochTrial, DwellTimes_ForBaseline, timeTyping, scoresDifficulty, subjName):
    
    pupilData['RLCorrelation'] = []
    pupilLogL = list()
    pupilLogR = list()

    # create a list of time of gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # Create list of pupil sizes from gazelog
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL, pupilLogR = Convert2ColumnsToFormPupilSizes(GazeLog)
          
    pupilL_avg = list()
    pupilR_avg = list()
    timeOfGaze_TrialList = list()
    
    pupilSizeL_Relative_allTrials = list()
    pupilSizeR_Relative_allTrials = list()
    scoresList = list()
    
    pupilSizeL_Absolute_allTrials = list()
    pupilSizeR_Absolute_allTrials = list()
    SamplesForBaselineList = list()
    SamplesForTrialList = list()
    
    pupilAverageL_ForTrialList = list()
    pupilAverageR_ForTrialList = list()
        
    # compute relative pupil size from the trial times
    # for every epoch, plot the pupil size
    for trialNr in range(0, len(timeEpochTrial['Start'])):
        if trialNr == 0:
            continue
        #print(trialNr)
        # find pupil sizes for the trial
        pupilSizeL_Trial, pupilSizeR_Trial, timeGaze_Trial, timeInternal_Trial = PupilSizeFromTrialTimes(
            [TimeEpochTrial['Start'][trialNr], TimeEpochTrial['End'][trialNr]], timeGazeLog, 
                                timeInternalGazeLog, pupilLogL, pupilLogR)
        
        pupilSize_Trial = dict()
        pupilSize_Filter = dict()
        pupilSize_woBlink = dict()
        
        # find difference in consecutive elements of internal time
        timeInternalDifference = [t - s for s, t in zip(timeInternal_Trial, timeInternal_Trial[1:])]
        # divide by 1000 to make it s
        timeOfGaze_Trial = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

        # some trials were skipped, because the sentence was written before. If the time of trial is less than
        # 10s, the trial is skipped
        if timeOfGaze_Trial[-1] < 20:
            print('trial number ', trialNr+1, 'with', timeOfGaze_Trial[-1], 's will be skipped')
            continue
        
        pupilSize_Trial['Left'] = pupilSizeL_Trial
        pupilSize_Trial['Right'] = pupilSizeR_Trial
        
        #if trialNr == 4:
        #    for i in range(0, len(pupilSizeL_Trial)):
        #        print(pupilSizeL_Trial[i], pupilSizeR_Trial[i])
            
        #print('Trial', len(pupilSizeL_Trial), len(pupilSizeR_Trial))
        
        # filter the blinks
        pupilSizeL_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeL_Trial, timeGaze_Trial)
        
        pupilSizeR_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeR_Trial, timeGaze_Trial)
        
        
        
        #print(index_blinkEndL)
        #print(index_blinkEndR)
        pupilSize_woBlink['Left'] = pupilSizeL_woBlink
        pupilSize_woBlink['Right'] = pupilSizeR_woBlink
        
        #print('After blink', len(pupilSizeL_woBlink), len(pupilSizeR_woBlink))
        # Hampel filter to remove the outliers
        winSize = 25
        pupilSizeL_filter = hampel(pupilSizeL_woBlink, winSize, 3)
        pupilSizeR_filter = hampel(pupilSizeR_woBlink, winSize, 3)

        pupilSize_Filter['Left'] = pupilSizeL_filter.values.tolist()
        pupilSize_Filter['Right'] = pupilSizeR_filter.values.tolist()
        
        pupilSizeL_filterList = [i[0] for i in pupilSizeL_filter.values]
        pupilSizeR_filterList = [i[0] for i in pupilSizeR_filter.values]
        
        #print('filter', len(pupilSizeL_filterList), len(pupilSizeR_filterList))
        RLCorrelation = np.corrcoef(pupilSizeL_filterList, pupilSizeR_filterList)
        
        if RLCorrelation[0][1] < 0.8:
            print(RLCorrelation[0][1])
            print('CORRELATION BETWEEN RIGHT AND LEFT IS NOT GOOD. TRIAL MUST BE REMOVED')
        
        # Relative Pupil Size Calculation 
        # First find baseline pupil size, which is the time when looking at NextPhrase key
        Samples_ForBaseline = int((int(DwellTimes_ForBaseline[trialNr-1][:-2])*90)/1000) # Number of samples of looking at key depend on
        
        
        # dwell time
        pupilL_baseline = np.mean(pupilSizeL_filterList[0:Samples_ForBaseline])
        pupilR_baseline = np.mean(pupilSizeR_filterList[0:Samples_ForBaseline])
        
        if relative_difference:
            pupilL_Relative = [pupil - pupilL_baseline for pupil in pupilSizeL_filterList]
            pupilR_Relative = [pupil - pupilR_baseline for pupil in pupilSizeR_filterList]
        else:
            pupilL_Relative = [(pupil - pupilL_baseline)/pupilL_baseline for pupil in pupilSizeL_filterList]
            pupilR_Relative = [(pupil - pupilR_baseline)/pupilR_baseline for pupil in pupilSizeR_filterList]
        
        #print(pupilL_baseline)
        
        # average of whole trial
        pupilL_avg.append(np.mean(pupilL_Relative))
        pupilR_avg.append(np.mean(pupilR_Relative))
        
        #print(np.mean(pupilL_Relative), np.mean(pupilR_Relative))
        
        pupilSize_Relative = dict()
        pupilSize_Relative['Left'] = pupilL_Relative
        pupilSize_Relative['Right'] = pupilR_Relative
    
        
        pupilSizeL_Relative_allTrials.extend(pupilL_Relative)
        pupilSizeR_Relative_allTrials.extend(pupilR_Relative)
        
        pupilSizeL_Absolute_allTrials.append(pupilSizeL_filterList)
        pupilSizeR_Absolute_allTrials.append(pupilSizeR_filterList)
        
        len_samplesBaseline = 0
        if len(SamplesForBaselineList) > 0:
            SamplesForBaselineList.append(Samples_ForBaseline + len(SamplesForTrialList))
            
        else:
            SamplesForBaselineList = [Samples_ForBaseline]
        
        # samples where average needs to be plotted
        SamplesForTrialList.extend(range(len(SamplesForTrialList), len(SamplesForTrialList)+len(pupilL_Relative)))
        
        # average for the trial - extended to the length of the trial for plot
        pupilAverageL_ForTrialList.extend([np.mean(pupilL_Relative)]*len(pupilL_Relative))
        pupilAverageR_ForTrialList.extend([np.mean(pupilR_Relative)]*len(pupilR_Relative))
        
        
        
        
        
        
        
        scoresList.append(scoresDifficulty.values[trialNr-1])
    
    plotRelativePupilSize_allTrials_wAvg(pupilSizeL_Relative_allTrials, pupilSizeR_Relative_allTrials, 
                                         pupilAverageL_ForTrialList, pupilAverageR_ForTrialList, SamplesForTrialList,
                                         SamplesForBaselineList, scoresList, subjName)
    
    
    


In [133]:
dataPath = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['RLCorrelation'] = []

# extract self-reported scores list and LIX score of given sentence
file_name = r'C:/DTU/Data/201812_ExptToCheckMovementEffect/Data/Scores.xlsx'

for root, dirs, subfolder in os.walk(dataPath):
    if not dirs and subj_session_name in root:
        
        if 'tb' in root or 'trial' in root:
            continue
            
        userKeys = None
        gazeLog = None
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_looks*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
            
            elif fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])

                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or keysSelected is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                if subjName == 'sa\Test_woChinRest\p2':
                    userKeys = userKeys[:-1]
                
                testNr_re = re.compile('(?<=\\\Test)(.*)')
                testNr = 'Test' + testNr_re.findall(subjName)[0]
                if 'rh\Test_wChinRest' in subjName or 'sa\Test_woChinRest' in subjName:
                    testNr = 'Test' + testNr_re.findall(subjName)[0][:-3]
                print(testNr)
                
                sheet_to_df_map = pd.read_excel(file_name, sheet_name=subjName[0:2])
                columnName = testNr + '_' + scoreType
                
                # find total of difficulty score
                scoresDifficulty = sheet_to_df_map[columnName]
                scoresDifficulty = scoresDifficulty[1:]
                
                #print(scoresDifficulty)
                
                # fix userKeys due to comma related file changes
                userKeys_new = FixUserKeys(userKeys)
                
                # find dwell time of typing
                userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
                
                # find start time of typing
                timeTyping = OptiKeyTypingTime(userKeys_wDwellTime)
                
                # for some of the subjects, the data was not completely collected
                if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
                    del keysSelected[-1]
                
                # divide complete data into epochs of phrases
                timeStartEndMixed = FindTrialEndTimes(keysSelected, timeTyping)
                
                # create trial time epoch using the list of start/end times of trial and userKeys, to make sure that 
                # Sleep is completely there in every trial, to allow for baseline
                timeEpochTrial = CreateTimeEpochsOfTrials(timeStartEndMixed, userKeys_wDwellTime)
                #print(timeEpochTrial)
                
                dwellTimes_ForBaseline = DwellTimeForBaseline(userKeys_wDwellTime)
                
                # find and plot pupil size for every trial
                pupilSize_CompleteSignal_wRelativeAvgOfTrial(gazeLog, timeEpochTrial, dwellTimes_ForBaseline, timeTyping, 
                                                                     scoresDifficulty, subjName)
                
                

sa\Test_woChinRest\p1
Test_woChinRest
sa\Test_woChinRest\p2
Test_woChinRest
trial number  3 with 8.449478 s will be skipped
trial number  5 with 8.926912 s will be skipped


In [116]:
def plotCompletePupilSize(pupilData, timeData, subjName):
    
    fig = plt.figure()
    axL = fig.add_subplot(2,1,1)
    axL.plot(timeData, pupilData['LeftRaw'][0:len(timeData)], 'b', label = 'Before blink removal')
    axL.plot(timeData, pupilData['LeftFiltered'][0:len(timeData)], 'r', label = 'After blink removal')
    plt.legend()
    
    axR = fig.add_subplot(2,1,2)
    axR.plot(timeData, pupilData['RightRaw'][0:len(timeData)], 'b', label = 'Before blink removal')
    axR.plot(timeData, pupilData['RightFiltered'][0:len(timeData)], 'r', label = 'After blink removal')
    plt.legend()

    axL.set_title('Left')
    #axL.set_xlabel('Time [in s]')
    axR.set_title('Right')
    axR.set_xlabel('Time [in s]')

    if absoluteSize:
        axR.set_ylabel('Absolute pupil size [in mm]')
        axL.set_ylabel('Absolute pupil size [in mm]')
    else:
        axR.set_ylabel('Relative pupil size [in %]')
        axL.set_ylabel('Relative pupil size [in %]')

    #axL.set_title(subjName)
    #plt.rcParams.update({'font.size': 20})
    
    pickle.dump(fig, open(r'C:\DTU\Results\201812_PilotExptForMovement\pupil Size\Complete pupil size' + '\\' + subj_session_name_fig , 'wb'))


In [ ]:
print(len(pupilData['LeftFiltered']))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(timeData, pupilData['LeftRaw'][0:len(timeData)], 'b')
ax.plot(timeData, pupilData['RightRaw'][0:len(timeData)], 'r')
ax.legend('Before blink removal', 'After blink removal')
    
ax.set_title('Left')
ax.set_xlabel('Time [in s]')
if absoluteSize:
    ax.set_ylabel('Absolute pupil size [in mm]')
else:
    ax.set_ylabel('Relative pupil size [in %]')
    
ax.set_title(subjName)
plt.rcParams.update({'font.size': 22})

In [ ]:
a = [2, 4, 1]
b = [4, 6, 3]

c = [float(str(a[i])+'.'+str(b[i])) for i in range(0, len(a))]

In [ ]:
c